In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Datasets/Churn_Modelling.csv")

In [ ]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [ ]:
df.drop(['RowNumber', 'CustomerId', 'Surname'],axis = 1, inplace = True)

In [ ]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [ ]:
df['Geography'].unique()

array(['France', 'Spain', 'Germany'], dtype=object)

In [ ]:
df = pd.get_dummies(df, columns = ['Geography', 'Gender'], drop_first=True)

In [ ]:
df.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,1,0
2,502,42,8,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,1,0


In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler()

min_max_df = scaler.fit_transform(df.drop("Exited", axis = 1))
min_max_df = pd.DataFrame(min_max_df,
                          columns = list(df.columns[:8]) + list(df.columns[9:]))

In [ ]:
min_max_df.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain,Gender_Male
0,0.538,0.324324,0.2,0.000000,0.000000,1.0,1.0,0.506735,0.0,0.0,0.0
1,0.516,0.310811,0.1,0.334031,0.000000,0.0,1.0,0.562709,0.0,1.0,0.0
2,0.304,0.324324,0.8,0.636357,0.666667,1.0,0.0,0.569654,0.0,0.0,0.0
3,0.698,0.283784,0.1,0.000000,0.333333,0.0,0.0,0.469120,0.0,0.0,0.0
4,1.000,0.337838,0.2,0.500246,0.000000,1.0,1.0,0.395400,0.0,1.0,0.0


In [ ]:
X = min_max_df
y = df['Exited']

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
oversample = SMOTE()

X, y = oversample.fit_resample(X, y)

X.shape, y.shape

((15926, 11), (15926,))

In [ ]:
y.value_counts()

1    7963
0    7963
Name: Exited, dtype: int64

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33,
                                                    random_state=42)

In [ ]:
tf.random.set_seed(42)

model = tf.keras.Sequential([
                             tf.keras.layers.Dense(10, activation = 'relu',
                                                   kernel_initializer='he_normal'),
                             tf.keras.layers.Dense(50, activation = 'relu',
                                                   kernel_initializer='he_normal'),
                             tf.keras.layers.Dense(100, activation = 'relu',
                                                   kernel_initializer='he_normal'),
                             tf.keras.layers.Dense(50, activation = 'relu',
                                                   kernel_initializer='he_normal'),
                             tf.keras.layers.Dense(10, activation = 'relu',
                                                   kernel_initializer='he_normal'),
                             tf.keras.layers.Dense(1, activation = 'sigmoid',
                                                   kernel_initializer='glorot_uniform')
])

history = model.compile(loss = tf.keras.losses.binary_crossentropy,
              optimizer = tf.keras.optimizers.Adam(),
              metrics = ['accuracy'])

model.fit(X,y, epochs = 100, verbose = 2)

Epoch 1/100
498/498 - 1s - loss: 0.6273 - accuracy: 0.6475 - 1s/epoch - 3ms/step
Epoch 2/100
498/498 - 1s - loss: 0.5728 - accuracy: 0.7020 - 832ms/epoch - 2ms/step
Epoch 3/100
498/498 - 1s - loss: 0.5475 - accuracy: 0.7235 - 814ms/epoch - 2ms/step
Epoch 4/100
498/498 - 1s - loss: 0.5215 - accuracy: 0.7400 - 842ms/epoch - 2ms/step
Epoch 5/100
498/498 - 1s - loss: 0.4868 - accuracy: 0.7620 - 826ms/epoch - 2ms/step
Epoch 6/100
498/498 - 1s - loss: 0.4617 - accuracy: 0.7744 - 848ms/epoch - 2ms/step
Epoch 7/100
498/498 - 1s - loss: 0.4496 - accuracy: 0.7856 - 843ms/epoch - 2ms/step
Epoch 8/100
498/498 - 1s - loss: 0.4439 - accuracy: 0.7881 - 848ms/epoch - 2ms/step
Epoch 9/100
498/498 - 1s - loss: 0.4367 - accuracy: 0.7932 - 855ms/epoch - 2ms/step
Epoch 10/100
498/498 - 1s - loss: 0.4335 - accuracy: 0.7934 - 797ms/epoch - 2ms/step
Epoch 11/100
498/498 - 1s - loss: 0.4292 - accuracy: 0.7975 - 806ms/epoch - 2ms/step
Epoch 12/100
498/498 - 1s - loss: 0.4238 - accuracy: 0.8003 - 844ms/epoch - 2

In [ ]:
preds = model.predict(X_test)

In [ ]:
preds = np.array([0 if i < 0.5 else 1 for i in preds])

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

In [ ]:
confusion_matrix(y_test, preds)

array([[2318,  344],
       [ 114, 2480]])

In [ ]:
accuracy_score(y_test, preds)

0.912861491628615

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       0.95      0.87      0.91      2662
           1       0.88      0.96      0.92      2594

    accuracy                           0.91      5256
   macro avg       0.92      0.91      0.91      5256
weighted avg       0.92      0.91      0.91      5256

